In [1]:
# -*- coding: utf-8 -*-
%matplotlib inline

import pandas as pd
import numpy as np
import math
import matplotlib as mplstyle
import matplotlib.pyplot as plt
import locale
from locale import atof
from dateutil import parser
from datetime import datetime

plt.style.use('fivethirtyeight')

### Import data

In [2]:
df_vouchersBySchool = pd.read_csv('school_choice_data/imported_data/awards_by_school_16.csv')
df_specialEdEnrollment = pd.read_csv('school_choice_data/gov_data/EnrollmentEnglishLearnerSpecEdSchls_ALL.csv')
df_missingSpecialEdEnrollment = pd.read_csv('school_choice_data/imported_data/updatedEnrollment.csv')
df_vouchersByOgSchoolDistrict = pd.read_csv('school_choice_data/imported_data/voucher-students-by-original-school-district.csv')
df_publicSpecialEdEnrollment = pd.read_csv('school_choice_data/gov_data/PublicEnrollmentEnglishLearnerSpecEd.csv')
df_schoolGrades2015 = pd.read_csv('school_choice_data/imported_data/final-2016-af-school-grades.csv')

In [3]:
# Added by Steve
df_specialEdEnrollment.head()

CORP                        CORP_NAME SCHL_NAME_CODE  \
0    15  Adams Central Community Schools            NaN   
1    15  Adams Central Community Schools             13   
2    15  Adams Central Community Schools             20   
3    15  Adams Central Community Schools             21   
4    25    North Adams Community Schools            NaN   

                         SCHL_NAME  ENR_ELL_N ENR_ELL_RATIO  ENR_SPEC_ED_N  \
0                              NaN         35         2.78%            132   
1      Adams Central Middle School          5         1.86%             16   
2  Adams Central Elementary School         22         3.59%             88   
3        Adams Central High School          8         2.12%             28   
4                              NaN         10         0.55%            260   

  ENR_SPEC_ED_RATIO  ENROLLMENT_N  
0            10.48%          1259  
1             5.95%           269  
2            14.38%           612  
3             7.41%           378  
4            14.37%          1809

In [4]:
# Added by Steve
df_missingSpecialEdEnrollment.head()

School ID  SpecEd 2016 SpecEd Perc  Enrollment 2016
0      C275         30.0       2.52%           1189.0
1      B009        114.0      11.26%           1012.0
2      B045         19.0       2.56%            743.0
3      B771          2.0       0.28%            709.0
4      D485         37.0       5.40%            685.0

### Functions

In [5]:
# replace symbols 
# use 0 for no school choice students and 9 for schools with less than 10 students
def replaceSymbols(id):
    if id == '*':
        return 9
    elif id == '-':
        return 0
    else:
        return id

# normalize numbers
# if is not a number, return None. if is a number, turn it into an integer
def normalizeNums(num):
    if (math.isnan(num)):
        return None
    else:
        num = int(math.floor(num))
        return num
    
# multiply by 100 to convert from percent
def convertPercent(num):
    return num * 100

# normalize percent from x% to just a number
def normalizePerc(num):
    try:
        float(num.strip('%'))
    except AttributeError:
        return num
    return float(num.strip('%'))
    
# get average award amount
# divide award amount by award count if they are both numbers
def averageAward(awardCount, awardAmount):
    if math.isnan(awardAmount) or math.isnan(awardCount):
        return None
    return round((awardAmount/awardCount),2)

# Convert NaN to zeros
# the zeros do not always indicate that no students 
# were in the program. schools with less than ten 
# students do not have to report 
def convertInt(num):
    if math.isnan(num):
        return 0
    else:
        return int(num)
    
# calculate percent
def calcPerc(num, denom):
    if num != 0:
        percent = (float(num/denom))*100
        return round(percent,2)
    else:
        return None
    
# calculate percent change
def percentChange(oldVal, newVal):
    val1 = float(oldVal)
    val2 = float(newVal)
    change = (val2-val1)/val1
    return float(change)

# fill in missing values
# compares two values, takes the first if is not null. if it is, takes second.
def fillInfo(choiceCell, backupCell):
    if pd.isnull(choiceCell):
        return backupCell
    elif math.isnan(choiceCell):
        return backupCell
    else:
        return choiceCell
    
# replace with None
def fillNone(num):
    if num == 1:
        return None
    else:
        return num 

### Voucher enrollment data

Apply function to replace symbols. Drop data from all but most recent school year. Filter out schools with no voucher enrollment.


In [6]:
# Added by Steve
df_vouchersBySchool.head()

School No.                     School Name      County 2011-2012 2012-2013  \
0       A260         Aboite Christian School  Huntington         -         -   
1       C905  Adventist Christian Elementary      Monroe         -         -   
2       A515      All Saints Catholic School        Cass         *        19   
3       B806    Ambassador Christian Academy        Lake       110       156   
4       C250       Anderson Christian School     Madison         -        12   

  2013-2014 2014-2015 2015-2016 2016-2017  
0         *        27        36        35  
1         -         *        10         *  
2        55        68         -         -  
3       257       206       210       154  
4        30        24        37        40

In [7]:
df_vouchersBySchool2016_filtered = df_vouchersBySchool
df_vouchersBySchool2016_filtered['Voucher enrollment 2016'] = df_vouchersBySchool2016_filtered['2016-2017'].apply(replaceSymbols)

# Trim out data from 2011-2015.
df_vouchersBySchool2016_trimmed = df_vouchersBySchool.drop(['2011-2012','2012-2013','2013-2014','2014-2015','2015-2016','2016-2017'], axis=1)

# Filter out schools with no voucher enrollment in 2016
# In the data, 1 == "-", and 9 == "Less than 10 voucher students"
# We want to filter out the "-" ones because they're basically zero.
df_vouchersBySchool2016_trimmed = df_vouchersBySchool2016_trimmed[df_vouchersBySchool2016_trimmed['Voucher enrollment 2016'] > 0]


In [8]:
# Added by Steve
df_vouchersBySchool2016_trimmed.head()

School No.                     School Name      County  \
0       A260         Aboite Christian School  Huntington   
1       C905  Adventist Christian Elementary      Monroe   
3       B806    Ambassador Christian Academy        Lake   
4       C250       Anderson Christian School     Madison   
5       B760             Andrean High School        Lake   

  Voucher enrollment 2016  
0                      35  
1                       9  
3                     154  
4                      40  
5                     161

### Indiana school enrollment (all)



In [9]:
# INDIANA SCHOOL ENROLLMENT (ALL)

# Filter out the public schools.
# Public school IDs are numbers-only. Private school IDs have letters + numbers.
def filterPrivate(id):
    if pd.isnull(id):
        return True
    else:
        try:
            float(id)
        except ValueError:
            return False
        return True

df_specialEdEnrollment_filtered = df_specialEdEnrollment
df_specialEdEnrollment_filtered['filter'] = df_specialEdEnrollment_filtered['SCHL_NAME_CODE'].apply(filterPrivate)
df_specialEdEnrollment_filtered = df_specialEdEnrollment_filtered[df_specialEdEnrollment_filtered['filter'] == False]
df_specialEdEnrollment_filtered = df_specialEdEnrollment_filtered.reset_index()

# Tidy and rename columns
df_specialEdEnrollment_trimmed = df_specialEdEnrollment_filtered.drop(['index','CORP','CORP_NAME','ENR_ELL_N','ENR_ELL_RATIO', 'filter'], axis=1)
df_specialEdEnrollment_trimmed = df_specialEdEnrollment_trimmed.rename(columns = {'SCHL_NAME_CODE':'School No.','SCHL_NAME':'School Name','ENR_SPEC_ED_N':'Special ed enrollment 2016','ENR_SPEC_ED_RATIO':'Special ed ratio 2016','ENROLLMENT_N':'Total enrollment 2016'})
df_specialEdEnrollment_trimmed['Special ed ratio 2016'] = df_specialEdEnrollment_trimmed['Special ed ratio 2016'].apply(normalizePerc)
df_specialEdEnrollment_trimmed

# Tidy and rename updated enrollment columns
df_missingSpecialEdEnrollment = df_missingSpecialEdEnrollment.rename(columns = {'School ID':'School No.','SpecEd 2016':'Special ed enrollment 2016','SpecEd Perc':'Special ed ratio 2016','Enrollment 2016':'Total enrollment 2016'})
df_missingSpecialEdEnrollment['Special ed ratio 2016'] = df_missingSpecialEdEnrollment['Special ed ratio 2016'].apply(normalizePerc)
df_missingSpecialEdEnrollment

# Merge w/ other special education data
df_specialEdEnrollment_merged = df_specialEdEnrollment_trimmed.merge(df_missingSpecialEdEnrollment,on='School No.', how='outer')
df_specialEdEnrollment_merged['Special ed enrollment 2016'] = np.vectorize(fillInfo)(df_specialEdEnrollment_merged['Special ed enrollment 2016_x'],df_specialEdEnrollment_merged['Special ed enrollment 2016_y'])
df_specialEdEnrollment_merged['Special ed ratio 2016'] = np.vectorize(fillInfo)(df_specialEdEnrollment_merged['Special ed ratio 2016_x'],df_specialEdEnrollment_merged['Special ed ratio 2016_y'])
df_specialEdEnrollment_merged['Total enrollment 2016'] = np.vectorize(fillInfo)(df_specialEdEnrollment_merged['Total enrollment 2016_x'],df_specialEdEnrollment_merged['Total enrollment 2016_y'])
df_specialEdEnrollment_merged = df_specialEdEnrollment_merged.drop(['Special ed enrollment 2016_x','Special ed enrollment 2016_y','Special ed ratio 2016_x','Special ed ratio 2016_y','Total enrollment 2016_x','Total enrollment 2016_y'], axis=1)
df_specialEdEnrollment_merged
# df_specialEdEnrollment_merged.to_csv('school_choice_data/exported_data/df_specialEdEnrollment_merged.csv')

# Merge w/ voucher school data
df_vouchersSpecialEd_merged = df_vouchersBySchool2016_trimmed.merge(df_specialEdEnrollment_merged,on='School No.', how='outer')
df_vouchersSpecialEd_merged = df_vouchersSpecialEd_merged.drop(['School Name_y'], axis=1)
df_vouchersSpecialEd_merged = df_vouchersSpecialEd_merged.rename(columns = {'School Name_x':'School Name'})
# df_vouchersSpecialEd_merged


In [10]:
# Added by Steve
df_missingSpecialEdEnrollment.head()

School No.  Special ed enrollment 2016  Special ed ratio 2016  \
0       C275                        30.0                   2.52   
1       B009                       114.0                  11.26   
2       B045                        19.0                   2.56   
3       B771                         2.0                   0.28   
4       D485                        37.0                   5.40   

   Total enrollment 2016  
0                 1189.0  
1                 1012.0  
2                  743.0  
3                  709.0  
4                  685.0

### School grades

In [11]:
# Associate schools with their current grade by merging grades df and df_vouchersSpecialEd_merged

# Tidy and rename columns
df_schoolGrades2015_trimmed = df_schoolGrades2015.drop(['IDOE_CORPORATION_ID','CORPORATION_NAME'], axis=1)
df_schoolGrades2015_trimmed = df_schoolGrades2015_trimmed.rename(columns = {'IDOE_SCHOOL_ID':'School No.','SCHOOL_NAME':'School Name','2015 Grade':'2015_grade'})
df_schoolGrades2015_trimmed

# Merge w/ voucher school data
df_vouchersSpecialEdGrades_merged = df_vouchersSpecialEd_merged.merge(df_schoolGrades2015_trimmed,on='School No.', how='outer')
df_vouchersSpecialEdGrades_merged = df_vouchersSpecialEdGrades_merged.drop(['School Name_y'], axis=1)
df_vouchersSpecialEdGrades_merged = df_vouchersSpecialEdGrades_merged.rename(columns = {'School Name_x':'School Name'})
df_vouchersSpecialEdGrades_merged

# Filter out all schools without vouchers in 2016
df_vouchersSpecialEdGrades_merged = df_vouchersSpecialEdGrades_merged[df_vouchersSpecialEdGrades_merged['Voucher enrollment 2016'] > 0]
# df_vouchersSpecialEdGrades_merged

### School districts

In [12]:
# Assign schools to parent school districts

# Tidy and rename
df_vouchersByOgSchoolDistrict_filtered = df_vouchersByOgSchoolDistrict.rename(columns = {'Corp. No.':'District No.','Corporation Name':'District Name'})
df_vouchersByOgSchoolDistrict_filtered = df_vouchersByOgSchoolDistrict_filtered.drop(['Voucher students from this district (2016-2017)'], axis=1)
# df_vouchersByOgSchoolDistrict_filtered

# Merge w/ school ID
df_vouchersSpecialEdGradesDistricts_merged = df_vouchersSpecialEdGrades_merged.merge(df_vouchersByOgSchoolDistrict_filtered,on='School No.', how='outer')
df_vouchersSpecialEdGradesDistricts_merged = df_vouchersSpecialEdGradesDistricts_merged.drop(['School Name_y'], axis=1)
df_vouchersSpecialEdGradesDistricts_merged = df_vouchersSpecialEdGradesDistricts_merged.rename(columns = {'School Name_x':'School Name'})
df_vouchersSpecialEdGradesDistricts_merged

df_vouchersSpecialEdGradesDistricts_merged.to_csv('school_choice_data/exported_data/df_vouchersSpecialEdGradesDistricts_merged.csv')

### Aggregate by parent school district

In [13]:
# Drop columns, group by district and reset index. 
df_districtVoucherSpecialEdAggregate = df_vouchersSpecialEdGradesDistricts_merged.drop(['Special ed ratio 2016'], axis=1)
df_districtVoucherSpecialEdAggregate = df_districtVoucherSpecialEdAggregate.groupby('District No.').sum()
df_districtVoucherSpecialEdAggregate = df_districtVoucherSpecialEdAggregate.reset_index()
df_districtVoucherSpecialEdAggregate

# Recalculate special ed ratio
df_districtVoucherSpecialEdAggregate['Special ed ratio 2016'] = np.vectorize(calcPerc)(df_districtVoucherSpecialEdAggregate['Special ed enrollment 2016'],df_districtVoucherSpecialEdAggregate['Total enrollment 2016'])
# df_districtVoucherSpecialEdAggregate

### Public school special ed enrollment 

In [14]:
# Merge in public school special ed enrollment data

# Tidy and clean columns
df_publicSpecialEdEnrollment_trimmed = df_publicSpecialEdEnrollment
df_publicSpecialEdEnrollment_trimmed = df_publicSpecialEdEnrollment_trimmed.drop(['ENR_ELL_N','ENR_ELL_RATIO'], axis=1)
df_publicSpecialEdEnrollment_trimmed = df_publicSpecialEdEnrollment_trimmed.rename(columns = {'CORP':'District No.','CORP_NAME':'District Name','ENR_SPEC_ED_N':'Public special ed enrollment 2016','ENR_SPEC_ED_RATIO':'Public special ed ratio 2016','ENROLLMENT_N':'Public total enrollment 2016'})
df_publicSpecialEdEnrollment_trimmed['Public special ed ratio 2016'] = df_publicSpecialEdEnrollment_trimmed['Public special ed ratio 2016'].apply(normalizePerc)
df_publicSpecialEdEnrollment_trimmed

# Merge
df_districtVoucherSpecialEdAggregate_merged = df_publicSpecialEdEnrollment_trimmed.merge(df_districtVoucherSpecialEdAggregate,on='District No.', how='outer')
df_districtVoucherSpecialEdAggregate_merged

# df_districtVoucherSpecialEdAggregate_merged.to_csv('school_choice_data/exported_data/df_districtVoucherSpecialEdAggregate_merged.csv')

District No.                       District Name  \
0              15     Adams Central Community Schools   
1              25       North Adams Community Schools   
2              35                 South Adams Schools   
3             125  M S D Southwest Allen County Schls   
4             225      Northwest Allen County Schools   
5             235        Fort Wayne Community Schools   
6             255           East Allen County Schools   
7             365         Bartholomew Con School Corp   
8             370      Flat Rock-Hawcreek School Corp   
9             395        Benton Community School Corp   
10            515            Blackford County Schools   
11            615       Western Boone Co Com Sch Dist   
12            630        Zionsville Community Schools   
13            665       Lebanon Community School Corp   
14            670     Brown County School Corporation   
15            750    Carroll Consolidated School Corp   
16            755        Delphi Community School Corp   
17            775        Pioneer Regional School Corp   
18            815            Southeastern School Corp   
19            875       Logansport Community Sch Corp   
20            940        West Clark Community Schools   
21           1000   Clarksville Community School Corp   
22           1010        Greater Clark County Schools   
23           1125              Clay Community Schools   
24           1150  Clinton Central School Corporation   
25           1160  Clinton Prairie School Corporation   
26           1170      Community Schools of Frankfort   
27           1180       Rossville Con School District   
28           1300  Crawford County Community Sch Corp   
29           1315    Barr-Reeve Community Schools Inc   
..            ...                                 ...   
361          9840   Excel Center - University Heights   
362          9845         Xavier School of Excellence   
363          9855          Excel Center - Noblesville   
364          9865        Hoosier Acad Virtual Charter   
365          9870            Discovery Charter School   
366          9875        Rock Creek Community Academy   
367          9880          Career Academy High School   
368          9885                 Gary Middle College   
369          9890              Indiana Virtual School   
370          9895   IN Math & Science Academy - North   
371          9900           Excel Center - South Bend   
372          9905         Indiana Connections Academy   
373          9910     Excel Center for Adult Learners   
374          9915                      Marion Academy   
375          9920               Damar Charter Academy   
376          9925  Phalen Leadership Academy - IN Inc   
377          9930       Nexus Academy of Indianapolis   
378          9935                      Vision Academy   
379          9940          Tindley Collegiate Academy   
380          9945         Tindley Renaissance Academy   
381          9950  Dugger Union Community School Corp   
382          9955              Mays Community Academy   
383          9960      Success Academy Primary School   
384          9965        Career Academy Middle School   
385          9970             ACE Preparatory Academy   
386          9975          Global Preparatory Academy   
387          9980                  Steel City Academy   
388          9985         Seven Oaks Classical School   
389          9990   Heritage Institute of Arts & Tech   
390          9995          Excel Center - Shelbyville   

     Public special ed enrollment 2016  Public special ed ratio 2016  \
0                                  132                         10.48   
1                                  260                         14.37   
2                                  195                         15.02   
3                                  565                          7.86   
4                                  887                         12.05   
5                                 4514